# Lab 2: Snapshots and Clones

In this lab, we'll leverage a `VolumeSnapshotClass` to create `VolumeSnapshots` that we later can use as a `.spec.dataSource` in a `PersistentVolumeClaim`. We'll also create a clone from an already existing `PersistentVolumeClaim`.

Let's begin by inspecting the cluster for `VolumeSnapshotClasses`.

In [ ]:
kubectl get volumesnapshotclasses

We'll use the "hpe-snapshot" `VolumeSnapshotClass` for these exercises. Let's examine it a bit further.

In [ ]:
kubectl get volumesnapshotclasses/hpe-snapshot -o yaml

Again, the most common scenario would be a `VolumeSnapshotClass` marked "default" on the cluster. Since this is a shared environment, we'll call the "hpe-snapshot" `VolumeSnapshotClass` explicitly in the following exercises.

# Deploy a workload

In this example, we'll use a single replica Redis database with persistence for our example application. We want to store and retrieve data from the database in the simplest way possible to illustrate how to use snapshots.

Assume the following `Deployment`.

---
```yaml
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-redis-pvc
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 320Mi
  storageClassName: hpe-standard
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-redis-deploy
  labels:
    app: my-redis-deploy
spec:
  replicas: 1
  strategy:
    type: Recreate
  selector:
    matchLabels:
      app: my-redis
  template:
    metadata:
      labels:
        app: my-redis
    spec:
      containers:
      - image: redis
        name: redis
        args: ["--appendonly", "yes"]
        volumeMounts:
          - name: my-data
            mountPath: /data
      volumes:
      - name: my-data
        persistentVolumeClaim:
          claimName: my-redis-pvc
```
---

We'll create everything in one swift stroke.

In [ ]:
kubectl create -f obj/my-redis-deploy.yaml
kubectl rollout status deploy/my-redis-deploy

Wait for the deployment to be rolled out. Then, let's create a new key.

In [ ]:
kubectl exec -it deploy/my-redis-deploy -- redis-cli set hpe/dev "This CSI workshop is awesome!"

Let’s make sure we can retrieve the key before proceeding.

In [ ]:
kubectl exec -it deploy/my-redis-deploy -- redis-cli get hpe/dev

## Create your first VolumeSnapshot

Creating a `VolumeSnapshot` is not more difficult than simply declaring it.

---
```yaml
---
apiVersion: snapshot.storage.k8s.io/v1beta1
kind: VolumeSnapshot
metadata:
  name: my-snapshot
spec:
  volumeSnapshotClassName: hpe-snapshot
  source:
    persistentVolumeClaimName: my-redis-pvc
```
---

Go ahead and create it.

In [ ]:
kubectl create -f obj/my-snapshot.yaml

A few seconds later, there should be a `VolumeSnapshot` named "my-snapshot" available for use in the `Namespace`.

In [ ]:
kubectl get -f obj/my-snapshot.yaml

## Create a new PersistentVolumeClaim from a VolumeSnapshot

A clone deployment has been prepared to illustrate how to use the `VolumeSnapshot`. In this case, we've separated the `PersistentVolumeClaim` from the `Deployment`.

---
```yaml
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-clone-pvc
spec:
  dataSource:
    name: my-snapshot
    kind: VolumeSnapshot
    apiGroup: snapshot.storage.k8s.io
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 320Mi
  storageClassName: hpe-standard
```
---

Pay attention to `.spec.dataSource`. This dictates that a `VolumeSnapshot` is going to be used as a source for the `PersistentVolumeClaim`.

Create the `PersistentVolumeClaim`.

**Note:** The `PersistentVolumeClaim` `.spec.resources.requests.storage` must match the parent `VolumeSnapshot` "RESTORESIZE".

In [ ]:
kubectl create -f obj/my-snapshot-pvc.yaml

Now, let's declare a `Deployment` that references the new `PersistentVolumeClaim`.

---
```yaml
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-clone-deploy
  labels:
    app: my-clone-deploy
spec:
  replicas: 1
  strategy:
    type: Recreate
  selector:
    matchLabels:
      app: my-clone
  template:
    metadata:
      labels:
        app: my-clone
    spec:
      containers:
      - image: redis
        name: redis
        args: ["--appendonly", "yes"]
        volumeMounts:
          - name: my-data
            mountPath: /data
      volumes:
      - name: my-data
        persistentVolumeClaim:
          claimName: my-clone-pvc
```
---

This `Deployment` is identical to the one deployed in the first step. Names have been changed to avoid collision and we can also see that we're referencing the "my-clone-pvc" `PersistentVolumeClaim`.

Create the `Deployment`.

In [ ]:
kubectl create -f obj/my-clone-deploy.yaml
kubectl rollout status deploy/my-clone-deploy

Let's make sure our key inserted into the parent `Deployment` is still intact.

In [ ]:
kubectl exec -it deploy/my-clone-deploy -- redis-cli get hpe/dev

You should see the same message inserted into the parent Redis store.

Deployments and persistent storage is completely decoupled. It's possible to modify the clone and the parent will be unaffected.

Let's delete our key in the clone and try retrieve it.

In [ ]:
kubectl exec -it deploy/my-clone-deploy -- redis-cli del hpe/dev
kubectl exec -it deploy/my-clone-deploy -- redis-cli get hpe/dev

Retrive the key from the parent.

In [ ]:
kubectl exec -it deploy/my-redis-deploy -- redis-cli get hpe/dev

Now, let's clean up a few things and redo this using the parent `PersistentVolumeClaim` directly.

In [ ]:
kubectl delete -f obj/my-clone-deploy.yaml -f obj/my-snapshot.yaml -f obj/my-snapshot-pvc.yaml

## Create a new PersistentVolumeClaim from another PersistentVolumeClaim

It's possible to skip the intermediary `VolumeSnapshot` and create a new `PersistentVolumeClaim` directly from an existing claim.

---
```yaml
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-clone-pvc
spec:
  dataSource:
    name: my-redis-pvc
    kind: PersistentVolumeClaim
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 320Mi
  storageClassName: hpe-standard
```
---

We don't need to specify the `.spec.dataSource.apiGroup` when `.spec.dataSource.kind` is `PersistentVolumeClaim`.

Let's create the claim.

In [ ]:
kubectl create -f obj/my-clone-pvc.yaml

Now, let's bring up the exact same `Deployment` as declared in the `VolumeSnapshot` case (we assume this has been deleted from the cluster).

In [ ]:
kubectl create -f obj/my-clone-deploy.yaml
kubectl rollout status deploy/my-clone-deploy

The key we inserted into the parent should be intact.

In [ ]:
kubectl exec -it deploy/my-clone-deploy -- redis-cli get hpe/dev

## Cleaning up

Run the following command to free up resources.

In [ ]:
kubectl delete deploy/my-clone-deploy deploy/my-redis-deploy pvc/my-clone-pvc pvc/my-redis-pvc

## Summary

The idea behind this is to keep `VolumeSnapshots` around and use it to recreate known good states of a stateful application. If just a recent instantiation of an application is needed adhoc, it's more convenient to use the parent `PersistentVolumeClaim` as a `.spec.dataSource`.

Let's continue on to [Lab 3: Ephemeral Inline Volumes](3-WKSHP-CSI-Inline.ipynb)

> **Learn more**
> - {{ BRANDING }} Community Blog: [HPE CSI Driver for Kubernetes: Snapshots, Clones and Volume Expansion](https://developer.hpe.com/blog/PklOy39w8NtX6M2RvAxW/hpe-csi-driver-for-kubernetes-snapshots-clones-and-volume-expansion)